In [2]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 5.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('train.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
df = df.dropna(subset=['question1', 'question2'])

In [6]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['is_duplicate'])

In [7]:
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['is_duplicate'])

In [8]:
import re

def preprocess(text):
    if pd.isnull(text):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

for df in [train_df, val_df, test_df]:
    df['question1'] = df['question1'].apply(preprocess)
    df['question2'] = df['question2'].apply(preprocess)


In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
from sentence_transformers import SentenceTransformer
model_name = 'all-MiniLM-L6-v2'

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [12]:
from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation
from torch.utils.data import DataLoader
import torch

train_examples = [InputExample(texts=[row['question1'], row['question2']], label=float(row['is_duplicate']))
                  for index, row in train_df.iterrows()]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

bi_encoder = SentenceTransformer(model_name, device='cuda' if torch.cuda.is_available() else 'cpu')

loss = losses.CosineSimilarityLoss(model=bi_encoder)

evaluator = evaluation.BinaryClassificationEvaluator(
    sentences1=val_df['question1'].tolist(),
    sentences2=val_df['question2'].tolist(),
    labels=val_df['is_duplicate'].astype(float).tolist()
)

bi_encoder.fit(
    train_objectives=[(train_dataloader, loss)],
    evaluator=evaluator,
    epochs=1,
    evaluation_steps=1000,
    warmup_steps=100,
    output_path='output/bi_encoder_cosine',
    use_amp=True  
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Cosine Accuracy,Cosine Accuracy Threshold,Cosine F1,Cosine F1 Threshold,Cosine Precision,Cosine Recall,Cosine Ap,Dot Accuracy,Dot Accuracy Threshold,Dot F1,Dot F1 Threshold,Dot Precision,Dot Recall,Dot Ap,Manhattan Accuracy,Manhattan Accuracy Threshold,Manhattan F1,Manhattan F1 Threshold,Manhattan Precision,Manhattan Recall,Manhattan Ap,Euclidean Accuracy,Euclidean Accuracy Threshold,Euclidean F1,Euclidean F1 Threshold,Euclidean Precision,Euclidean Recall,Euclidean Ap,Max Accuracy,Max Accuracy Threshold,Max F1,Max F1 Threshold,Max Precision,Max Recall,Max Ap
1000,0.139400,No log,0.823864,0.608741,0.778616,0.565268,0.710582,0.861057,0.809335,0.823864,0.608741,0.778616,0.565268,0.710582,0.861057,0.809481,0.824482,13.769339,0.777662,14.285698,0.716098,0.850807,0.809278,0.823864,0.884601,0.778616,0.932451,0.710582,0.861057,0.809335,0.824482,13.769339,0.778616,14.285698,0.716098,0.861057,0.809481
2000,0.131500,No log,0.834500,0.624634,0.788207,0.563832,0.728352,0.858779,0.827210,0.834500,0.624634,0.788207,0.563832,0.728352,0.858779,0.827358,0.834005,13.083944,0.787751,14.402431,0.732355,0.852214,0.827102,0.834500,0.866448,0.788207,0.933990,0.728352,0.858779,0.827210,0.834500,13.083944,0.788207,14.402431,0.732355,0.858779,0.827358
3000,0.124400,No log,0.842687,0.615121,0.798525,0.560673,0.737733,0.870235,0.839943,0.842687,0.615122,0.798525,0.560673,0.737733,0.870235,0.839812,0.842737,13.505814,0.798264,14.406073,0.743058,0.862330,0.839850,0.842687,0.877358,0.798525,0.937366,0.737733,0.870235,0.839943,0.842737,13.505814,0.798525,14.406073,0.743058,0.870235,0.839943
4000,0.122900,No log,0.846867,0.633864,0.802154,0.580140,0.749070,0.863335,0.846559,0.846867,0.633864,0.802154,0.580140,0.749070,0.863335,0.846506,0.846694,13.003998,0.802105,14.363365,0.745441,0.868091,0.846472,0.846867,0.855728,0.802154,0.916362,0.749070,0.863335,0.846559,0.846867,13.003998,0.802154,14.363365,0.749070,0.868091,0.846559
5000,0.120000,No log,0.850281,0.633182,0.805311,0.574952,0.749899,0.869565,0.851162,0.850281,0.633182,0.805311,0.574952,0.749899,0.869565,0.851186,0.850157,13.362518,0.805708,14.255762,0.753102,0.866216,0.851047,0.850281,0.856525,0.805311,0.922007,0.749899,0.869565,0.851162,0.850281,13.362518,0.805708,14.255762,0.753102,0.869565,0.851186
6000,0.119700,No log,0.852358,0.638371,0.810005,0.582600,0.760284,0.866685,0.853934,0.852358,0.638371,0.810005,0.582600,0.760284,0.866685,0.853995,0.852408,13.297185,0.810160,14.174862,0.761540,0.865412,0.853843,0.852358,0.850445,0.810005,0.913674,0.760284,0.866685,0.853934,0.852408,13.297185,0.810160,14.174862,0.761540,0.866685,0.853995
7000,0.115600,No log,0.853274,0.641741,0.810531,0.586685,0.761574,0.866216,0.856864,0.853274,0.641741,0.810531,0.586685,0.761574,0.866216,0.856886,0.853076,13.390553,0.810767,14.279335,0.757789,0.871709,0.856782,0.853274,0.846474,0.810531,0.909192,0.761574,0.866216,0.856864,0.853274,13.390553,0.810767,14.279335,0.761574,0.871709,0.856886
8000,0.113300,No log,0.854312,0.637162,0.813042,0.567774,0.755887,0.879547,0.858150,0.854312,0.637162,0.813042,0.567775,0.755887,0.879547,0.858213,0.854560,13.422560,0.812377,14.546280,0.752873,0.882093,0.858045,0.854312,0.851866,0.813042,0.929759,0.755887,0.879547,0.858150,0.854560,13.422560,0.813042,14.546280,0.755887,0.882093,0.858213
9000,0.114800,No log,0.854807,0.626865,0.812749,0.568649,0.757718,0.876398,0.858691,0.854807,0.626865,0.812749,0.568649,0.757718,0.876398,0.858920,0.854634,12.910105,0.812626,14.171444,0.766277,0.864943,0.858604,0.854807,0.863869,0.812749,0.928817,0.757718,0.876398,0.858691,0.854807,12.910105,0.812749,14.171444,0.766277,0.876398,0.858920
10000,0.114400,No log,0.855599,0.631488,0.813064,0.566381,0.758869,0.875595,0.859405,0.855599,0.631488,0.813064,0.566381,0.758869,0.875595,0.859349,0.855401,13.187016,0.812794,14.301113,0.764016,0.868225,0.859307,0.855599,0.858501,0.813064,0.931256,0.758869,0.875595,0.859405,0.855599,13.187016,0.813064,14.301113,0.764016,0.875595,0.859405

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [13]:
from sentence_transformers.losses import ContrastiveLoss

contrastive_loss = ContrastiveLoss(model=bi_encoder)

bi_encoder.fit(
    train_objectives=[(train_dataloader, contrastive_loss)],
    evaluator=evaluator,
    epochs=1,
    evaluation_steps=1000,
    warmup_steps=100,
    output_path='output/bi_encoder_contrastive',
    use_amp=True
)


Step,Training Loss,Validation Loss,Cosine Accuracy,Cosine Accuracy Threshold,Cosine F1,Cosine F1 Threshold,Cosine Precision,Cosine Recall,Cosine Ap,Dot Accuracy,Dot Accuracy Threshold,Dot F1,Dot F1 Threshold,Dot Precision,Dot Recall,Dot Ap,Manhattan Accuracy,Manhattan Accuracy Threshold,Manhattan F1,Manhattan F1 Threshold,Manhattan Precision,Manhattan Recall,Manhattan Ap,Euclidean Accuracy,Euclidean Accuracy Threshold,Euclidean F1,Euclidean F1 Threshold,Euclidean Precision,Euclidean Recall,Euclidean Ap,Max Accuracy,Max Accuracy Threshold,Max F1,Max F1 Threshold,Max Precision,Max Recall,Max Ap
1000,0.012500,No log,0.868634,0.815661,0.827606,0.772353,0.779145,0.882495,0.875703,0.868634,0.815662,0.827606,0.772353,0.779145,0.882495,0.875716,0.868189,9.461376,0.827612,10.563437,0.776512,0.885911,0.875620,0.868634,0.607188,0.827606,0.674755,0.779145,0.882495,0.875703,0.868634,9.461376,0.827612,10.563437,0.779145,0.885911,0.875716
2000,0.012500,No log,0.870341,0.811458,0.831503,0.768637,0.776351,0.895089,0.879548,0.870341,0.811458,0.831503,0.768637,0.776351,0.895089,0.879350,0.870217,9.464338,0.831498,10.622085,0.774387,0.897702,0.879433,0.870341,0.614071,0.831503,0.680240,0.776351,0.895089,0.879548,0.870341,9.464338,0.831503,10.622085,0.776351,0.897702,0.879548
3000,0.012200,No log,0.874694,0.803666,0.837867,0.775873,0.783926,0.899779,0.883505,0.874694,0.803666,0.837867,0.775873,0.783926,0.899779,0.883322,0.874743,9.617493,0.837593,10.462124,0.782333,0.901253,0.883422,0.874694,0.626632,0.837867,0.669517,0.783926,0.899779,0.883505,0.874743,9.617493,0.837867,10.462124,0.783926,0.901253,0.883505
4000,0.011600,No log,0.875560,0.802413,0.838087,0.779295,0.793711,0.887720,0.886592,0.875560,0.802413,0.838087,0.779295,0.793711,0.887720,0.886764,0.875832,9.815351,0.838244,10.641382,0.782561,0.902459,0.886533,0.875560,0.628629,0.838087,0.664387,0.793711,0.887720,0.886592,0.875832,9.815351,0.838244,10.641382,0.793711,0.902459,0.886764
5000,0.011900,No log,0.877019,0.803816,0.841085,0.781862,0.795449,0.892276,0.887481,0.877019,0.803816,0.841085,0.781862,0.795449,0.892276,0.887535,0.877044,9.509559,0.841043,10.485172,0.787863,0.901923,0.887470,0.877019,0.626393,0.841085,0.660512,0.795449,0.892276,0.887481,0.877044,9.509559,0.841085,10.485172,0.795449,0.901923,0.887535
6000,0.011600,No log,0.878651,0.799904,0.841640,0.785661,0.800326,0.887452,0.889829,0.878651,0.799904,0.841640,0.785661,0.800326,0.887452,0.889903,0.878107,9.845848,0.841873,10.040579,0.805496,0.881691,0.889845,0.878651,0.632608,0.841640,0.654735,0.800326,0.887452,0.889829,0.878651,9.845848,0.841873,10.040579,0.805496,0.887452,0.889903
7000,0.011600,No log,0.879765,0.816348,0.843940,0.797419,0.804826,0.887050,0.890817,0.879765,0.816348,0.843940,0.797419,0.804826,0.887050,0.891089,0.879542,9.283292,0.843849,10.140768,0.794989,0.899109,0.890817,0.879765,0.606056,0.843940,0.636523,0.804826,0.887050,0.890817,0.879765,9.283292,0.843940,10.140768,0.804826,0.899109,0.891089
8000,0.011300,No log,0.880655,0.806673,0.845270,0.780293,0.799474,0.896630,0.892171,0.880655,0.806673,0.845270,0.780293,0.799474,0.896630,0.892066,0.880408,9.880693,0.845082,10.452726,0.793426,0.903932,0.892145,0.880655,0.621815,0.845270,0.662884,0.799474,0.896630,0.892171,0.880655,9.880693,0.845270,10.452726,0.799474,0.903932,0.892171
9000,0.011500,No log,0.880828,0.805342,0.845698,0.778597,0.794617,0.903798,0.893047,0.880828,0.805342,0.845698,0.778597,0.794617,0.903798,0.893071,0.880853,9.765572,0.845435,10.321465,0.795137,0.902526,0.893000,0.880828,0.623952,0.845698,0.665437,0.794617,0.903798,0.893047,0.880853,9.765572,0.845698,10.321465,0.795137,0.903798,0.893071
10000,0.011400,No log,0.880878,0.801594,0.846253,0.776190,0.797475,0.901387,0.893696,0.880878,0.801594,0.846253,0.776190,0.797475,0.901387,0.893816,0.880803,9.918058,0.846285,10.333284,0.800119,0.898104,0.893667,0.880878,0.629929,0.846253,0.669045,0.797475,0.901387,0.893696,0.880878,9.918058,0.846285,10.333284,0.800119,0.901387,0.893816


In [14]:
from sentence_transformers.losses import MultipleNegativesRankingLoss

positive_train_df = train_df[train_df['is_duplicate'] == 1]

train_examples_mnrl = [InputExample(texts=[row['question1'], row['question2']])
                       for index, row in positive_train_df.iterrows()]

train_dataloader_mnrl = DataLoader(train_examples_mnrl, shuffle=True, batch_size=32)

mnrl_loss = MultipleNegativesRankingLoss(model=bi_encoder)

bi_encoder.fit(
    train_objectives=[(train_dataloader_mnrl, mnrl_loss)],
    evaluator=evaluator,
    epochs=1,
    evaluation_steps=1000,
    warmup_steps=100,
    output_path='output/bi_encoder_mnrl',
    use_amp=True
)


Step,Training Loss,Validation Loss,Cosine Accuracy,Cosine Accuracy Threshold,Cosine F1,Cosine F1 Threshold,Cosine Precision,Cosine Recall,Cosine Ap,Dot Accuracy,Dot Accuracy Threshold,Dot F1,Dot F1 Threshold,Dot Precision,Dot Recall,Dot Ap,Manhattan Accuracy,Manhattan Accuracy Threshold,Manhattan F1,Manhattan F1 Threshold,Manhattan Precision,Manhattan Recall,Manhattan Ap,Euclidean Accuracy,Euclidean Accuracy Threshold,Euclidean F1,Euclidean F1 Threshold,Euclidean Precision,Euclidean Recall,Euclidean Ap,Max Accuracy,Max Accuracy Threshold,Max F1,Max F1 Threshold,Max Precision,Max Recall,Max Ap
1000,0.020900,No log,0.853496,0.795935,0.814339,0.768413,0.748736,0.892544,0.855852,0.853496,0.795935,0.814339,0.768413,0.748736,0.892544,0.855958,0.852927,9.849564,0.814518,10.568998,0.749746,0.891539,0.855734,0.853496,0.638851,0.814339,0.680569,0.748736,0.892544,0.855852,0.853496,9.849564,0.814518,10.568998,0.749746,0.892544,0.855958
2000,0.019800,No log,0.849737,0.804366,0.808926,0.775373,0.749971,0.877939,0.850831,0.849737,0.804366,0.808926,0.775373,0.749971,0.877939,0.850842,0.849588,9.681833,0.807875,10.513803,0.744938,0.882428,0.850732,0.849737,0.625514,0.808926,0.670264,0.749971,0.877939,0.850831,0.849737,9.681833,0.808926,10.513803,0.749971,0.882428,0.850842
3000,0.021800,No log,0.850652,0.808593,0.809071,0.781741,0.750760,0.877202,0.850642,0.850652,0.808593,0.809071,0.781741,0.750760,0.877202,0.850916,0.850627,9.682549,0.808911,10.248860,0.751567,0.875729,0.850541,0.850652,0.618720,0.809071,0.660694,0.750760,0.877202,0.850642,0.850652,9.682549,0.809071,10.248860,0.751567,0.877202,0.850916
3732,0.018500,No log,0.850380,0.808482,0.807444,0.781889,0.748598,0.876331,0.848556,0.850380,0.808482,0.807444,0.781889,0.748598,0.876331,0.848671,0.850602,9.653419,0.807364,10.273513,0.747972,0.877001,0.848438,0.850380,0.618899,0.807444,0.660471,0.748598,0.876331,0.848556,0.850602,9.653419,0.807444,10.273513,0.748598,0.877001,0.848671


In [21]:
from sentence_transformers import CrossEncoder

cross_encoder_model_name = 'cross-encoder/quora-roberta-base'
cross_encoder = CrossEncoder(cross_encoder_model_name, num_labels=1, device='cuda' if torch.cuda.is_available() else 'cpu')

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [31]:
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator

# Prepare training data
train_samples = [(row['question1'], row['question2'], row['is_duplicate'])
                 for index, row in train_df.iterrows()]

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2",num_labels=1, device='cuda' if torch.cuda.is_available() else 'cpu')


NameError: name 'al_df' is not defined

In [35]:
evaluator = CEBinaryClassificationEvaluator(
    sentence_pairs=list(zip(val_df['question1'].tolist(), val_df['question2'].tolist())),
    labels=val_df['is_duplicate'].tolist()
)

from sentence_transformers import InputExample
from torch.utils.data import DataLoader

train_samples = [(row['question1'], row['question2'], row['is_duplicate'])
                 for index, row in train_df.iterrows()]

# Create a DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Train the model
cross_encoder.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=1,
    evaluation_steps=1000,
    warmup_steps=100,
    output_path='output/cross_encoder',
    use_amp=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20215 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def evaluate_model(model, test_df, is_cross_encoder=False):
    if not is_cross_encoder:
        embeddings1 = model.encode(test_df['question1'].tolist(), convert_to_tensor=True, show_progress_bar=True)
        embeddings2 = model.encode(test_df['question2'].tolist(), convert_to_tensor=True, show_progress_bar=True)
        cosine_scores = torch.nn.functional.cosine_similarity(embeddings1, embeddings2).cpu().numpy()
        threshold = 0.5
        predictions = [1 if score > threshold else 0 for score in cosine_scores]
    else:
        scores = model.predict(list(zip(test_df['question1'].tolist(), test_df['question2'].tolist())), show_progress_bar=True)
        scores = np.array(scores)
        probabilities = 1 / (1 + np.exp(-scores))
        threshold = 0.5
        predictions = [1 if prob > threshold else 0 for prob in probabilities]

    f1 = f1_score(test_df['is_duplicate'].tolist(), predictions)
    return f1

bi_encoder_base = SentenceTransformer(model_name, device='cuda' if torch.cuda.is_available() else 'cpu')
f1_base = evaluate_model(bi_encoder_base, test_df)

bi_encoder_cosine = SentenceTransformer('output/bi_encoder_cosine', device='cuda' if torch.cuda.is_available() else 'cpu')
f1_bi_cosine = evaluate_model(bi_encoder_cosine, test_df)

bi_encoder_contrastive = SentenceTransformer('output/bi_encoder_contrastive', device='cuda' if torch.cuda.is_available() else 'cpu')
f1_bi_contrastive = evaluate_model(bi_encoder_contrastive, test_df)

bi_encoder_mnrl = SentenceTransformer('output/bi_encoder_mnrl', device='cuda' if torch.cuda.is_available() else 'cpu')
f1_bi_mnrl = evaluate_model(bi_encoder_mnrl, test_df)

cross_encoder = CrossEncoder('output/cross_encoder', device='cuda' if torch.cuda.is_available() else 'cpu')
f1_cross = evaluate_model(cross_encoder, test_df, is_cross_encoder=True)

print(f"Off-the-shelf Bi-encoder (Unfine-tuned) F1-Score: {f1_base:.4f}")
print(f"Bi-encoder with Cosine Similarity Loss F1-Score: {f1_bi_cosine:.4f}")
print(f"Bi-encoder with Contrastive Loss F1-Score: {f1_bi_contrastive:.4f}")
print(f"Bi-encoder with Multiple Negative Ranking Loss F1-Score: {f1_bi_mnrl:.4f}")
print(f"Cross-encoder F1-Score: {f1_cross:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

Batches:   0%|          | 0/1264 [00:00<?, ?it/s]